<a href="https://colab.research.google.com/github/anjaleeDS/Sandbox/blob/master/Rose_WEBscrapingNotes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas way to scrape and clean UC Davis rose catalog

In [2]:
import lxml.etree as xml
import requests  # go out and fetch itnative python
import pandas as pd
import numpy as np
import re
import time
import requests


In [3]:
from google.colab import drive
drive.mount('drive')
%cd drive/My Drive/Colab Notebooks/webscraping


Mounted at drive
/content/drive/My Drive/Colab Notebooks/webscraping


In [4]:
url = "https://fps.ucdavis.edu/roses/collection.cfm"
requests.get(url)

<Response [200]>

## Get the table data

In [140]:
r=requests.get(url)
df_list= pd.read_html(r.text)


make into a dataframe

In [202]:
df = pd.DataFrame(data=df_list[1],copy=True)
df.shape

(821, 6)

clean the headers

In [203]:
df.columns = df.columns.str.strip() # blanks
df.columns = df.columns.str.replace(r"[^a-zA-Z\d\_]+", "")

df.columns


Index(['Unnamed0', 'Cultivar', 'FlowerColor', 'FlowerColor1', 'PlantType',
       'Fragrance'],
      dtype='object')

take out columns that have no data


In [204]:
for col in df.columns:
  if df[col].count()==0:
    # print(col)
    df = df.drop(col, axis=1)  

df.columns


Index(['Cultivar', 'FlowerColor1', 'PlantType', 'Fragrance'], dtype='object')

In [205]:
df.shape[1]

4

### get the detail page URLs

In [206]:
d_url = (re.findall('cultivar.cfm\?varietyid=\d+', r.text)) # get all the onclicks
d_url = pd.DataFrame(data=d_url) # stick them in a df

df = (pd.concat([df,d_url],axis=1)) # concat the dfs together
df = df.rename({df.columns[df.shape[1]-1] :'details'},axis=1) # rename that concated column to the correct column name

# make all of the details into urls with detail
df['details'] = df.apply(lambda x:"https://fps.ucdavis.edu/roses/"+  x['details'], axis=1)

df

,Cultivar,FlowerColor1,PlantType,Fragrance,details
0,A Shropshire Lad ®,light pink,Shrub,moderate,cultivar.cfm?varietyid=3568
1,Abbaye De Cluny ™,apricot & apricot blend,Hybrid Tea & Climbing Hybrid Tea,slight,cultivar.cfm?varietyid=2132
2,About Face ™,orange & orange blend,Grandiflora & Climbing Grandiflora,none,cultivar.cfm?varietyid=2774
3,Above All ™,orange & orange blend,Large-Flowered Climber,moderate - fruity,cultivar.cfm?varietyid=4973
4,Above and Beyond ™,apricot & apricot blend,Large-Flowered Climber,moderate - spice,cultivar.cfm?varietyid=6254
...,...,...,...,...,...
816,Young Lycidas,deep pink,Shrub,intense - delicious tea and old rose with hint...,cultivar.cfm?varietyid=6060
817,You're the One ™,pink blend,Miniature & Climbing Miniature,slight,cultivar.cfm?varietyid=5005
818,Yuletide,medium red,Hybrid Tea & Climbing Hybrid Tea,slight,cultivar.cfm?varietyid=1631
819,Yves Piaget ®,medium pink,Hybrid Tea & Climbing Hybrid Tea,intense,cultivar.cfm?varietyid=1704


## Fetch the details from the detail page

In [210]:
def add_details(detail_url, df):
  d = requests.get(detail_url)
  return (pd.read_html(d.text))
  

In [270]:
df['details'][0]

'https://fps.ucdavis.edu/roses/cultivar.cfm?varietyid=3568'

In [ ]:
first_record = add_details(df['details'][0], df) # get the things from the detail page
rose_records = pd.DataFrame(data=first_record)

for i in range(df.shape[0]):
  next_record = add_details(df['details'][i], df)
  rose_records[i] = (next_record[2][1])

rose_records

/usr/local/lib/python3.7/dist-packages/pandas/core/internals/construction.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  values = np.array([convert(v) for v in values])


In [287]:
deets_data = pd.DataFrame(data=deets_text[2]) # turn the useful things or data into a dataframe
deets2_data = pd.DataFrame(data=deets_text2[2])

In [294]:
deets_data[2] = deets2_data[1]
deets_data

,0,1,2
0,Cultivar Name:,A Shropshire Lad ®,Abbaye De Cluny ™
1,Synonyms:,'L/58/88','Romantic Seranade'
2,Patent:,David Austin Roses Limited PP10607,The Conard-Pyle Company PP09609
3,Description:,Introduction Year: 1996; sub-type: English Lea...,Meibrinpay
4,Registration Name:,Ausled,HT & Cl HT - Hybrid Tea & Climbing Hybrid Tea
5,Classification:,S - Shrub,No
6,Climber:,No,apricot & apricot blend
7,Petal Color:,light pink,Meilland International SA
8,Hybridizer:,"Austin, David","orange apricot, 4¼ in., full, borne mostly singly"
9,Flower Form:,"flowers strikingly beautiful soft peachy pink,...",slight


Get detail page columns

In [261]:
# change the current row into columns
rose_df = pd.DataFrame(columns=deets[0])
# clean the column names
rose_df.columns = rose_deet.columns.str.strip() # blanks
rose_df.columns = rose_deet.columns.str.replace(r"[^a-zA-Z\d\_]+", "")

rose_df.columns


Index(['CultivarName', 'Synonyms', 'Patent', 'Description', 'RegistrationName',
       'Classification', 'Climber', 'PetalColor', 'Hybridizer', 'FlowerForm',
       'Fragrance', 'Foliage', 'Prickle', 'Awards', 'References', 'Parentage'],
      dtype='object', name=0)

In [264]:
# place the deets in the final df
for i in range(df.shape[0]):
  print(df.shape[0] , 'i\'m on', i)

821 i'm on 0
821 i'm on 1
821 i'm on 2
821 i'm on 3
821 i'm on 4
821 i'm on 5
821 i'm on 6
821 i'm on 7
821 i'm on 8
821 i'm on 9
821 i'm on 10
821 i'm on 11
821 i'm on 12
821 i'm on 13
821 i'm on 14
821 i'm on 15
821 i'm on 16
821 i'm on 17
821 i'm on 18
821 i'm on 19
821 i'm on 20
821 i'm on 21
821 i'm on 22
821 i'm on 23
821 i'm on 24
821 i'm on 25
821 i'm on 26
821 i'm on 27
821 i'm on 28
821 i'm on 29
821 i'm on 30
821 i'm on 31
821 i'm on 32
821 i'm on 33
821 i'm on 34
821 i'm on 35
821 i'm on 36
821 i'm on 37
821 i'm on 38
821 i'm on 39
821 i'm on 40
821 i'm on 41
821 i'm on 42
821 i'm on 43
821 i'm on 44
821 i'm on 45
821 i'm on 46
821 i'm on 47
821 i'm on 48
821 i'm on 49
821 i'm on 50
821 i'm on 51
821 i'm on 52
821 i'm on 53
821 i'm on 54
821 i'm on 55
821 i'm on 56
821 i'm on 57
821 i'm on 58
821 i'm on 59
821 i'm on 60
821 i'm on 61
821 i'm on 62
821 i'm on 63
821 i'm on 64
821 i'm on 65
821 i'm on 66
821 i'm on 67
821 i'm on 68
821 i'm on 69
821 i'm on 70
821 i'm on 71
82

In [245]:

deets.iloc[15][1] # 15th element's data vs [15][0] is the header label

'Heritage x unnamed seedling'